In [27]:
import re
from typing import Optional, Dict


In [28]:
# Funçao que extrai o autor, ano e página das citações por meio de regex
def extrai_autor_ano_pagina(citacao: str) -> Dict[str, Optional[str]]:

    s = citacao.strip()

    if (s.startswith("(") and s.endswith(")")) or (s.startswith("[") and s.endswith("]")):
        s = s[1:-1].strip()

    s = s.rstrip(" ;,.")

    autor: Optional[str] = None
    ano: Optional[str] = None
    pagina: Optional[str] = None

    m_ano_colado = re.search(r"\b(?P<ano>\d{4}[a-z]?)(?:\s*[:]\s*(?P<pagina>\d+(?:[-–]\d+)?))", s)
    if m_ano_colado:
        ano = m_ano_colado.group("ano")
        pagina = m_ano_colado.group("pagina")
        autor_raw = s[:m_ano_colado.start()].strip()
        autor = autor_raw.rstrip(" ,(").lstrip("(")
        autor = re.sub(r"\s+", " ", autor) or None
        return {"autor": autor, "ano": ano, "pagina": pagina}

    m_ano = re.search(r"\b(\d{4}[a-z]?)\b", s)
    if m_ano:
        ano = m_ano.group(1)
        autor_raw = s[:m_ano.start()].strip()
        autor = autor_raw.rstrip(" ,(").lstrip("(")
        autor = re.sub(r"\s+", " ", autor) or None
    else:
        m_autor = re.match(r"([A-Za-zÀ-ÖØ-öø-ÿ][A-Za-zÀ-ÖØ-öø-ÿ\.\'\-]*(?:\s+[A-Za-zÀ-ÖØ-öø-ÿ][A-Za-zÀ-ÖØ-öø-ÿ\.\'\-]*)*)", s)
        autor = (m_autor.group(1).strip() if m_autor else None)

    start = m_ano.end() if m_ano else 0
    tail = s[start:]

    m_pag = re.search(r"\bpp?\.\s*(\d+(?:[-–]\d+)?)\b", tail)
    if not m_pag:
        m_pag = re.search(r":\s*(\d+(?:[-–]\d+)?)\b", tail)
    if not m_pag:
        m_pag = re.search(r",\s*(\d+(?:[-–]\d+)?)\b", tail)

    if m_pag:
        pagina = m_pag.group(1)

    return {"autor": autor, "ano": ano, "pagina": pagina}

print(extrai_autor_ano_pagina("Mises (1949, p.258)"))
print(extrai_autor_ano_pagina("(von Mises, 1963, p.254)"))
print(extrai_autor_ano_pagina("Mises, pp. 105-6;"))
print(extrai_autor_ano_pagina("(C Berg 2022)"))


{'autor': 'Mises', 'ano': '1949', 'pagina': '258'}
{'autor': 'von Mises', 'ano': '1963', 'pagina': '254'}
{'autor': 'Mises', 'ano': None, 'pagina': '105-6'}
{'autor': 'C Berg', 'ano': '2022', 'pagina': None}


In [29]:
# Função que testa a função extrai_autor_ano com uma série de citações em diferentes padrões
def teste_extrai_autor_ano():
    casos = [
        ("Mises (1949, p.258)", {"autor": "Mises", "ano": "1949", "pagina": "258"}),
        ("(Mises, 1996, pp. 538-86)", {"autor": "Mises", "ano": "1996", "pagina": "538-86"}),
        ("(von Mises, 1963, p.254)", {"autor": "von Mises", "ano": "1963", "pagina": "254"}),
        ("(Mises, 1920, 121-122)", {"autor": "Mises", "ano": "1920", "pagina": "121-122"}),
        ("(Mises 1949, 236-237)", {"autor": "Mises", "ano": "1949", "pagina": "236-237"}),
        ("(Mises 1920, 109)", {"autor": "Mises", "ano": "1920", "pagina": "109"}),
        ("(Mises 1920, p.162)", {"autor": "Mises", "ano": "1920", "pagina": "162"}),
        ("(von Mises, 1949: 351)", {"autor": "von Mises", "ano": "1949", "pagina": "351"}),
        ("(Mises 1966: 493)", {"autor": "Mises", "ano": "1966", "pagina": "493"}),
        ("Hayek (1976:71)", {"autor": "Hayek", "ano": "1976", "pagina": "71"}),
        ("(von Mises 1998, p. 270)", {"autor": "von Mises", "ano": "1998", "pagina": "270"}),
        ("Mises 1949, p. 3)", {"autor": "Mises", "ano": "1949", "pagina": "3"}),
        ("Mises 1985b, p. 236", {"autor": "Mises", "ano": "1985b", "pagina": "236"}),
        ("Mises 1957b, 372", {"autor": "Mises", "ano": "1957b", "pagina": "372"}),
        ("Mises, pp. 105-6;", {"autor": "Mises", "ano": None, "pagina": "105-6"}),
        ("(Mises, 1949, p. 3)", {"autor": "Mises", "ano": "1949", "pagina": "3"}),
        ("(L Von Mises 1949, pp. 393)", {"autor": "L Von Mises", "ano": "1949", "pagina": "393"}),
        ("(L Von Mises 1949 , pp. 393)", {"autor": "L Von Mises", "ano": "1949", "pagina": "393"}),
        ("(C Berg 2022)", {"autor": "C Berg", "ano": "2022", "pagina": None}),
    ]
    for texto, esperado in casos:
        resultado = extrai_autor_ano_pagina(texto)
        print(f"Entrada: {texto}")
        print(f"Resultado: {resultado}")
        print(f"Esperado: {esperado}")
        print(f"Passou? {resultado == esperado}")
        print("-" * 50)

teste_extrai_autor_ano()

Entrada: Mises (1949, p.258)
Resultado: {'autor': 'Mises', 'ano': '1949', 'pagina': '258'}
Esperado: {'autor': 'Mises', 'ano': '1949', 'pagina': '258'}
Passou? True
--------------------------------------------------
Entrada: (Mises, 1996, pp. 538-86)
Resultado: {'autor': 'Mises', 'ano': '1996', 'pagina': '538-86'}
Esperado: {'autor': 'Mises', 'ano': '1996', 'pagina': '538-86'}
Passou? True
--------------------------------------------------
Entrada: (von Mises, 1963, p.254)
Resultado: {'autor': 'von Mises', 'ano': '1963', 'pagina': '254'}
Esperado: {'autor': 'von Mises', 'ano': '1963', 'pagina': '254'}
Passou? True
--------------------------------------------------
Entrada: (Mises, 1920, 121-122)
Resultado: {'autor': 'Mises', 'ano': '1920', 'pagina': '121-122'}
Esperado: {'autor': 'Mises', 'ano': '1920', 'pagina': '121-122'}
Passou? True
--------------------------------------------------
Entrada: (Mises 1949, 236-237)
Resultado: {'autor': 'Mises', 'ano': '1949', 'pagina': '236-237'}
Esp